In [1]:
pwd

'/root/spark'

In [2]:
#filepath = "bydata/by-day/2010-12-01.csv"
filepath = "./bydata/by-day/2010-12-01.csv"

In [3]:
# read DataFrame
df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(filepath)
df.show(2)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows



In [4]:
df.createOrReplaceTempView("dfTable")

In [5]:
from pyspark.sql.functions import lit
df.select(lit(5), lit('five'), lit(2.0)).show(2)

+---+----+---+
|  5|five|2.0|
+---+----+---+
|  5|five|2.0|
|  5|five|2.0|
+---+----+---+
only showing top 2 rows



In [6]:
from pyspark.sql.functions import col

# InvoiceNo != 536365 InvoiceNo, Description만 출력하는데, 5개만
spark.sql("""
    select InvoiceNo,Description from dfTable where InvoiceNo != 536365 limit 5
""").show(truncate = False)

# truncate = False : 자동 생략 없이 전체가 출력됨

df.where(col("InvoiceNo") != 536365).select("InvoiceNo","Description").show(10,False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+

+---------+----------------------------------+
|InvoiceNo|Description                       |
+---------+----------------------------------+
|536366   |HAND WARMER UNION JACK            |
|536366   |HAND WARMER RED POLKA DOT         |
|536367   |ASSORTED COLOUR BIRD ORNAMENT     |
|536367   |POPPY'S PLAYHOUSE BEDROOM         |
|536367   |POPPY'S PLAYHOUSE KITCHEN         |
|536367   |FELTCRAFT PRINCESS CHARLOTTE DOLL |
|536367   |IVORY KNITTED MUG COSY            |
|536367   |BOX OF 6 ASSORTED COLOUR TEASPOONS|
|536367   |BOX OF VINTAGE JIGSAW BLOCKS      |
|536367   |BOX OF VINTAGE ALPHABET BLOCKS    |
+---------

In [7]:
# 1. UnitPrice 가격이 600 이상이고 
# 2. description POSTAGE 문자열 모함
# StockCode의 값중에 DOT인 행을 선택하는데 1번과 2번 조건 중 하나라도 만족
# sql 방식
spark.sql('''
   SELECT * FROM dfTable 
   WHERE StockCode = 'DOT' AND (UnitPrice >= 600 OR INSTR(Description, 'POSTAGE') >= 1)
''').show(5)

# 함수 사용 방식
from pyspark.sql.functions import instr
dotcondition = col('StockCode') == 'DOT'
pricecondition = col('UnitPrice') > 600
# descondition = instr(col("Decriotion"),'POSTAGE') >= 1
descondition = col('Description').isin('POSTAGE')
df.where(dotcondition).where(pricecondition | descondition).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      NULL|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      NULL|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      

In [8]:
# 조건을 만들고 조건에 맞는 새로운 데이터를 새로운 컬럼에 넣어서 추가
df.withColumn('isExpensive', dotcondition & (pricecondition | descondition)).show(2)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|isExpensive|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|      false|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|      false|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
only showing top 2 rows



In [9]:
# isExpensive True인 항목중에 UnitPrice, isExpensive 출력
df.withColumn('isExpensive', dotcondition & (pricecondition | descondition))\
.where(col('isExpensive') == 'true')\
.select('UnitPrice','isExpensive').show(2)

df.withColumn('isExpensive', dotcondition & (pricecondition | descondition))\
.where('isExpensive == true')\
.select('UnitPrice','isExpensive').show(2)

+---------+-----------+
|UnitPrice|isExpensive|
+---------+-----------+
|   607.49|       true|
+---------+-----------+

+---------+-----------+
|UnitPrice|isExpensive|
+---------+-----------+
|   607.49|       true|
+---------+-----------+



In [10]:
from pyspark.sql.functions import expr, pow

# (수량 가격)**2+5 --> 새로운 컬럼에 대입
new_col = pow(col('Quantity')*col('UnitPrice'),2)+5
df.select(expr('CustomerId'), new_col.alias('realQuantity')).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [11]:
from pyspark.sql.functions import round, bround, lit

df.select(round(lit('2.5')), bround(lit('2.5'))).show()

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 20 rows



In [12]:
df.select('UnitPrice').describe().show()

+-------+------------------+
|summary|         UnitPrice|
+-------+------------------+
|  count|              3108|
|   mean| 4.151946589446603|
| stddev|15.638659854603892|
|    min|               0.0|
|    max|            607.49|
+-------+------------------+



In [13]:
from pyspark.sql.functions import count, mean, min, max, stddev_pop
# stddev_pop : 표준 편차, 모집단위의 표준 편차 - 모든 개체에 대한 표준 편차, 즉 실제 표준 편차를 제공
# stddev : 표본의 표준편차

In [14]:
colname = 'UnitPrice'
quantileProbs = [0.5] # 사분위수 0.5 중위수를 나타냄
realError = 0.05
df.stat.approxQuantile(colname,quantileProbs,realError)

[2.51]

In [15]:
df.stat.crosstab('StockCode','Quantity').show(2)
# crosstab : 행렬
# df.select('StockCode','Quantity').show(10).stat.crosstab('StockCode','Quantity').show(2)

# 데이터프레임에 필요한 필드를 선택하고 출력합니다.
df.select('StockCode','Quantity').show(10)

# 출력된 데이터를 기반으로 크로스탭을 생성합니다.
df.stat.crosstab('StockCode','Quantity').show(2)

24/03/21 14:05:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288|  3| 30| 32| 33| 34| 36|384|  4| 40|432| 47| 48|480|  5| 50| 56|  6| 60|600| 64|  7| 70| 72|  8| 80|  9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|             21259|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  2|  0|  0|  0|  0| 

In [16]:
# 'StockCode','Quantity'10개의 데이터 중에서 df_stockcode_quantity_top10 10개만 추출한 data frame
df_stockcode_quantity_top10 = df.limit(10)
df_stockcode_quantity_top10.stat.crosstab("StockCode", "Quantity").show()

+------------------+---+---+---+---+
|StockCode_Quantity|  2| 32|  6|  8|
+------------------+---+---+---+---+
|            85123A|  0|  0|  1|  0|
|             71053|  0|  0|  1|  0|
|            84406B|  0|  0|  0|  1|
|            84029G|  0|  0|  1|  0|
|            84029E|  0|  0|  1|  0|
|             22752|  1|  0|  0|  0|
|             21730|  0|  0|  1|  0|
|             22633|  0|  0|  1|  0|
|             22632|  0|  0|  1|  0|
|             84879|  0|  1|  0|  0|
+------------------+---+---+---+---+



In [17]:
df_stockcode_quantity_top10.stat.freqItems(["StockCode","Quantity"]).show()

+--------------------+------------------+
| StockCode_freqItems|Quantity_freqItems|
+--------------------+------------------+
|[22752, 22632, 22...|     [32, 8, 2, 6]|
+--------------------+------------------+



In [18]:
df_stockcode_quantity_top10.stat.freqItems(["StockCode"]).show(truncate=False)

+--------------------------------------------------------------------------+
|StockCode_freqItems                                                       |
+--------------------------------------------------------------------------+
|[22752, 22632, 22633, 84879, 71053, 84406B, 85123A, 21730, 84029E, 84029G]|
+--------------------------------------------------------------------------+



In [19]:
df_stockcode_quantity_top10.select("StockCode","Quantity").show()

+---------+--------+
|StockCode|Quantity|
+---------+--------+
|   85123A|       6|
|    71053|       6|
|   84406B|       8|
|   84029G|       6|
|   84029E|       6|
|    22752|       2|
|    21730|       6|
|    22633|       6|
|    22632|       6|
|    84879|      32|
+---------+--------+



In [20]:
from pyspark.sql.functions import monotonically_increasing_id
df.select(monotonically_increasing_id()).show()

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
|                            2|
|                            3|
|                            4|
|                            5|
|                            6|
|                            7|
|                            8|
|                            9|
|                           10|
|                           11|
|                           12|
|                           13|
|                           14|
|                           15|
|                           16|
|                           17|
|                           18|
|                           19|
+-----------------------------+
only showing top 20 rows



In [21]:
# 첫 글자 대문자
from pyspark.sql.functions import initcap
df.select(initcap("Description")).show()

+--------------------+
|initcap(Description)|
+--------------------+
|White Hanging Hea...|
| White Metal Lantern|
|Cream Cupid Heart...|
|Knitted Union Fla...|
|Red Woolly Hottie...|
|Set 7 Babushka Ne...|
|Glass Star Froste...|
|Hand Warmer Union...|
|Hand Warmer Red P...|
|Assorted Colour B...|
|Poppy's Playhouse...|
|Poppy's Playhouse...|
|Feltcraft Princes...|
|Ivory Knitted Mug...|
|Box Of 6 Assorted...|
|Box Of Vintage Ji...|
|Box Of Vintage Al...|
|Home Building Blo...|
|Love Building Blo...|
|Recipe Box With M...|
+--------------------+
only showing top 20 rows



In [22]:
from pyspark.sql.functions import lower, upper
df.select(col('Description'), lower('Description'),lower('Description'),upper('Description')).show(2)

+--------------------+--------------------+--------------------+--------------------+
|         Description|  lower(Description)|  lower(Description)|  upper(Description)|
+--------------------+--------------------+--------------------+--------------------+
|WHITE HANGING HEA...|white hanging hea...|white hanging hea...|WHITE HANGING HEA...|
| WHITE METAL LANTERN| white metal lantern| white metal lantern| WHITE METAL LANTERN|
+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows



In [23]:
from pyspark.sql.functions import translate
df.select('Description',translate('Description','LEFT','1234')).show(2, truncate=False)

+----------------------------------+----------------------------------+
|Description                       |translate(Description, LEFT, 1234)|
+----------------------------------+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|WHI42 HANGING H2AR4 4-1IGH4 HO1D2R|
|WHITE METAL LANTERN               |WHI42 M24A1 1AN42RN               |
+----------------------------------+----------------------------------+
only showing top 2 rows



In [24]:
from pyspark.sql.functions import regexp_extract

# 정규 표현식을 사용하여 Description에서 색상과 관련된 단어를 추출합니다.
# extract_str는 BLACK, RED, WHITE, GREEN, BLUE와 같은 단어를 추출하는 정규 표현식입니다.
extract_str = "(BLACK|RED|WHITE|GREEN|BLUE)"
df.select(
    regexp_extract('Description', extract_str, 1).alias('color'),
    'Description'
).show(5)


+-----+--------------------+
|color|         Description|
+-----+--------------------+
|WHITE|WHITE HANGING HEA...|
|WHITE| WHITE METAL LANTERN|
|     |CREAM CUPID HEART...|
|     |KNITTED UNION FLA...|
|  RED|RED WOOLLY HOTTIE...|
+-----+--------------------+
only showing top 5 rows



In [25]:
# instr을 사용하여  Decriotion중에서 특정 문자열이 포함되어 있는지 여부를 확인하고 새로운 열을 추가
# 'BLACK','WHITE' ------>hasColorBW
containBlack = instr('Description','BLACK') >= 1
containWhite = instr('Description','WHITE') >= 1
df.withColumn('hasColorBW',containBlack|containWhite).show(2)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|hasColorBW|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|      true|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|      true|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+----------+
only showing top 2 rows



In [49]:
from pyspark.sql.functions import expr, locate
simpleColors = ["black", "white", "red", "green", "blue"]
def color_locator(column, color_string):
  return locate(color_string.upper(), column)\
          .cast("boolean")\
          .alias("is_" + color_string)
selectedColumns = [color_locator(df.Description, c) for c in simpleColors]
selectedColumns.append(expr("*")) # Column 타입이어야 합니다.

# unpacking
df.select(*selectedColumns).where(expr("is_white OR is_red"))\
  .select("Description").show(3, False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



In [27]:
# 날짜
from pyspark.sql.functions import current_date, current_timestamp
datedf = spark.range(10).withColumn('today', current_date())\
				.withColumn("now", current_timestamp())
datedf.show(truncate=False)
datedf.createOrReplaceTempView('datedf')

+---+----------+-----------------------+
|id |today     |now                    |
+---+----------+-----------------------+
|0  |2024-03-21|2024-03-21 14:08:41.449|
|1  |2024-03-21|2024-03-21 14:08:41.449|
|2  |2024-03-21|2024-03-21 14:08:41.449|
|3  |2024-03-21|2024-03-21 14:08:41.449|
|4  |2024-03-21|2024-03-21 14:08:41.449|
|5  |2024-03-21|2024-03-21 14:08:41.449|
|6  |2024-03-21|2024-03-21 14:08:41.449|
|7  |2024-03-21|2024-03-21 14:08:41.449|
|8  |2024-03-21|2024-03-21 14:08:41.449|
|9  |2024-03-21|2024-03-21 14:08:41.449|
+---+----------+-----------------------+



In [28]:
from pyspark.sql.functions import date_add, date_sub

In [32]:
datedf.select(date_sub('today', 5), date_add('today',5)).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2024-03-16|        2024-03-26|
+------------------+------------------+
only showing top 1 row



In [50]:
from pyspark.sql.functions import datediff,months_between,to_date
datedf.withColumn('week_ago',date_sub('today',7))\
        .select(datediff('week_ago','today')).show(3)

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
|                       -7|
|                       -7|
+-------------------------+
only showing top 3 rows



In [38]:
datedf.select(
    to_date(lit('2024-03-21')).alias('start'),
    to_date(lit('2024-05-08')).alias('end'))\
    .select(months_between('start','end')).show(1)

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                     -1.58064516|
+--------------------------------+
only showing top 1 row



In [51]:
from pyspark.sql.functions import to_date, lit
spark.range(1).withColumn('date',lit('2024-05-02')).select('date',to_date('date')).show()

+----------+-------------+
|      date|to_date(date)|
+----------+-------------+
|2024-05-02|   2024-05-02|
+----------+-------------+



In [52]:
dateformat = 'yyyy-dd-MM'
spark.range(1).select(to_date(lit('2024-05-02'), dateformat) ).show()

+-------------------------------+
|to_date(2024-05-02, yyyy-dd-MM)|
+-------------------------------+
|                     2024-02-05|
+-------------------------------+



In [53]:
from pyspark.sql.functions import coalesce
df.select(coalesce('Description','CustomerId')).show(truncate=False)

+-----------------------------------+
|coalesce(Description, CustomerId)  |
+-----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|
|RED WOOLLY HOTTIE WHITE HEART.     |
|SET 7 BABUSHKA NESTING BOXES       |
|GLASS STAR FROSTED T-LIGHT HOLDER  |
|HAND WARMER UNION JACK             |
|HAND WARMER RED POLKA DOT          |
|ASSORTED COLOUR BIRD ORNAMENT      |
|POPPY'S PLAYHOUSE BEDROOM          |
|POPPY'S PLAYHOUSE KITCHEN          |
|FELTCRAFT PRINCESS CHARLOTTE DOLL  |
|IVORY KNITTED MUG COSY             |
|BOX OF 6 ASSORTED COLOUR TEASPOONS |
|BOX OF VINTAGE JIGSAW BLOCKS       |
|BOX OF VINTAGE ALPHABET BLOCKS     |
|HOME BUILDING BLOCK WORD           |
|LOVE BUILDING BLOCK WORD           |
|RECIPE BOX WITH METAL HEART        |
+-----------------------------------+
only showing top 20 rows



In [54]:
data = [
    (None,2),
    (1,None),
    (1,2),
    (None,None),
]
example_df = spark.createDataFrame(data,['a','b'])
example_df.show()
example_df.select(coalesce('a','b')).show(truncate=False)

+----+----+
|   a|   b|
+----+----+
|NULL|   2|
|   1|NULL|
|   1|   2|
|NULL|NULL|
+----+----+

+--------------+
|coalesce(a, b)|
+--------------+
|2             |
|1             |
|1             |
|NULL          |
+--------------+



In [55]:
# StockCode, InvoiceNo가  NA 인 항목만 추출
# df.filter( "StockCode == 'None' or InvoiceNo == 'None'").show()
df.filter( col("StockCode").isNull() | col("InvoiceNo").isNull() ).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [61]:
example_df.filter(col('a').isNull() | col('b').isNull()).show()

+----+----+
|   a|   b|
+----+----+
|NULL|   2|
|   1|NULL|
|NULL|NULL|
+----+----+



In [ ]:
# na.drop : na항목만 drop
# na.fill : 채우다

In [62]:
example_df.na.drop('all', subset=['a']).show()

+---+----+
|  a|   b|
+---+----+
|  1|NULL|
|  1|   2|
+---+----+



In [63]:
example_df.na.drop('all', subset=['a','b']).show()

+----+----+
|   a|   b|
+----+----+
|NULL|   2|
|   1|NULL|
|   1|   2|
+----+----+



In [64]:
example_df.na.fill('all', subset=['a','b']).show()

+----+----+
|   a|   b|
+----+----+
|NULL|   2|
|   1|NULL|
|   1|   2|
|NULL|NULL|
+----+----+



In [65]:
fill_cols_vals = {'a':100,'b':5}
example_df.na.fill(fill_cols_vals).show()

+---+---+
|  a|  b|
+---+---+
|100|  2|
|  1|  5|
|  1|  2|
|100|  5|
+---+---+



In [56]:
# 컬럼끼리 묶기 : 복합 컬럼 만들기
from pyspark.sql.functions import struct
df.select( struct('Description','InvoiceNo').alias('complete')).show(truncate=False)

+---------------------------------------------+
|complete                                     |
+---------------------------------------------+
|{WHITE HANGING HEART T-LIGHT HOLDER, 536365} |
|{WHITE METAL LANTERN, 536365}                |
|{CREAM CUPID HEARTS COAT HANGER, 536365}     |
|{KNITTED UNION FLAG HOT WATER BOTTLE, 536365}|
|{RED WOOLLY HOTTIE WHITE HEART., 536365}     |
|{SET 7 BABUSHKA NESTING BOXES, 536365}       |
|{GLASS STAR FROSTED T-LIGHT HOLDER, 536365}  |
|{HAND WARMER UNION JACK, 536366}             |
|{HAND WARMER RED POLKA DOT, 536366}          |
|{ASSORTED COLOUR BIRD ORNAMENT, 536367}      |
|{POPPY'S PLAYHOUSE BEDROOM , 536367}         |
|{POPPY'S PLAYHOUSE KITCHEN, 536367}          |
|{FELTCRAFT PRINCESS CHARLOTTE DOLL, 536367}  |
|{IVORY KNITTED MUG COSY , 536367}            |
|{BOX OF 6 ASSORTED COLOUR TEASPOONS, 536367} |
|{BOX OF VINTAGE JIGSAW BLOCKS , 536367}      |
|{BOX OF VINTAGE ALPHABET BLOCKS, 536367}     |
|{HOME BUILDING BLOCK WORD, 536367}     

In [58]:
from pyspark.sql.functions import split
df.select(split("Description"," ")).show(2,False)

+----------------------------------------+
|split(Description,  , -1)               |
+----------------------------------------+
|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|
|[WHITE, METAL, LANTERN]                 |
+----------------------------------------+
only showing top 2 rows



In [59]:
df.select(split("Description"," ").alias('array_col'))\
    .selectExpr('array_col[0]').show(2)

+------------+
|array_col[0]|
+------------+
|       WHITE|
|       WHITE|
+------------+
only showing top 2 rows



In [60]:
from pyspark.sql.functions import size
df.select( size(split("Description"," ")).alias('array_size')).show(2)

+----------+
|array_size|
+----------+
|         5|
|         3|
+----------+
only showing top 2 rows



In [68]:
# 새로운 행으로 확장 explode
from pyspark.sql.functions import explode
df.withColumn('splitted',split("Description"," "))\
    .withColumn('exploded',explode('splitted'))\
    .select('Description','splitted','exploded').show(10,False)

+----------------------------------+----------------------------------------+--------+
|Description                       |splitted                                |exploded|
+----------------------------------+----------------------------------------+--------+
|WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|WHITE   |
|WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|HANGING |
|WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|HEART   |
|WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|T-LIGHT |
|WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|HOLDER  |
|WHITE METAL LANTERN               |[WHITE, METAL, LANTERN]                 |WHITE   |
|WHITE METAL LANTERN               |[WHITE, METAL, LANTERN]                 |METAL   |
|WHITE METAL LANTERN               |[WHITE, METAL, LANTERN]                 |LANTERN |
|CREAM CUPID HEARTS COAT HANGER    |[CREAM,

In [70]:
from pyspark.sql.functions import create_map
df.select(create_map('Description','InvoiceNo').alias('complex_ma')).show(2,False)

+----------------------------------------------+
|complex_ma                                    |
+----------------------------------------------+
|{WHITE HANGING HEART T-LIGHT HOLDER -> 536365}|
|{WHITE METAL LANTERN -> 536365}               |
+----------------------------------------------+
only showing top 2 rows



In [77]:
df.select(create_map('Description', 'InvoiceNo').alias('complex_ma'))\
    .selectExpr('complex_ma["WHITE METAL LANTERN"]').show()


+-------------------------------+
|complex_ma[WHITE METAL LANTERN]|
+-------------------------------+
|                           NULL|
|                         536365|
|                           NULL|
|                           NULL|
|                           NULL|
|                           NULL|
|                           NULL|
|                           NULL|
|                           NULL|
|                           NULL|
|                           NULL|
|                           NULL|
|                           NULL|
|                           NULL|
|                           NULL|
|                           NULL|
|                           NULL|
|                           NULL|
|                           NULL|
|                           NULL|
+-------------------------------+
only showing top 20 rows



In [75]:
# join type DataFrame
json_df = spark.range(1).selectExpr("""
    '{"myJSONKey" : {"myJSONValue" : [1,2,3]}}' as jsonString
""")

In [76]:
json_df.show(truncate = False)

+-----------------------------------------+
|jsonString                               |
+-----------------------------------------+
|{"myJSONKey" : {"myJSONValue" : [1,2,3]}}|
+-----------------------------------------+



In [79]:
# json : 키값만 가지고 분석간으
from pyspark.sql.functions import get_json_object, json_tuple
json_df.select(
    # $로 가져오거나 키:값으로 가져옴
    get_json_object('jsonString',"$.myJSONKey.myJSONValue[1]").alias('columns'),
    json_tuple('jsonString','myJSONKey')
).show(truncate=False)

+-------+-----------------------+
|columns|c0                     |
+-------+-----------------------+
|2      |{"myJSONValue":[1,2,3]}|
+-------+-----------------------+



In [83]:
# JSON 형태로 바꿀 수도 있다.
from pyspark.sql.functions import to_json
df.selectExpr("(InvoiceNo,Description) as myStruct").show(2,truncate=False)

+--------------------------------------------+
|myStruct                                    |
+--------------------------------------------+
|{536365, WHITE HANGING HEART T-LIGHT HOLDER}|
|{536365, WHITE METAL LANTERN}               |
+--------------------------------------------+
only showing top 2 rows



In [84]:
df.selectExpr("(InvoiceNo,Description) as myStruct")\
    .select(to_json("mystruct"))\
    .show(2,truncate=False)

+-------------------------------------------------------------------------+
|to_json(mystruct)                                                        |
+-------------------------------------------------------------------------+
|{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"}|
|{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}               |
+-------------------------------------------------------------------------+
only showing top 2 rows

